In [121]:
import numpy as np

In [122]:
# 定义全局常量
G = 9.81

# 定义Junction节点类
class Junction:
    def __init__(self):
        self.ki = 0 
        self.ko = 0
        self.flag = 0
        self.in_ = np.zeros(10,dtype=int)
        self.out = np.zeros(10,dtype=int)
        self.z = 0.0
        self.dz = 0.0
        self.o = 0.0
        self.a = 0.0

# 定义Channel河段类        
class Channel:
    def __init__(self):
        self.name = ''
        self.no = 0
        self.st = 0 
        self.en = 0
        self.m = 0
        self.slope = 0.0
        self.la = 0.0
        self.ds = 0.0
        self.s = np.zeros(1000)
        self.zb = np.zeros(1000)
        # ...省略其它成员变量初始化...
        
# 定义全局变量        
JuncNum = 0 
ChanNum = 0
ChanM = 100
SecN = 1000
JuncM = 100
JuncN = 10 

Chan = [Channel() for i in range(ChanM)] 
Junc = [Junction() for i in range(JuncM)]

In [123]:
# 定义变量
m = 1000 
theta = 0.75
alp = 1.0

In [124]:
# 定义文件夹路径
folder = '/home/louyaxin/1DHydroDynamicModel/input/'
folder1 = '/home/louyaxin/1DHydroDynamicModel/result/'

In [125]:
# 打开拓扑文件
with open(folder + 'TOPO.txt') as f:
    JuncNum, JuncNum1, JuncNum2 = [int(x) for x in f.readline().split()] 
    ChanNum, chanNum1, chanNum2 = [int(x) for x in f.readline().split()]
    UbouNum, AbouNum = [int(x) for x in f.readline().split()]
    
    print(f'ChanNum={ChanNum}, chanNum1={chanNum1}, chanNum2={chanNum2}') 
    print(f'JuncNum={JuncNum}, JuncNum1={JuncNum1}, JuncNum2={JuncNum2}')
    print('river star end')

ChanNum=3, chanNum1=0, chanNum2=1
JuncNum=4, JuncNum1=1, JuncNum2=2
river star end


In [126]:
# 定义Channel数组
Chan = np.zeros(ChanNum, dtype=object)
with open(folder + 'TOPO.txt') as f:
    for i in range(ChanNum):
        Chan[i] = [int(x) for x in f.readline().split()[:4]]
    print(Chan[i])
    
print('Handle the relationship between river section number and node number:')

[1, 3]
Handle the relationship between river section number and node number:


In [127]:
# 定义河道段类
class Channel:
    def __init__(self, no, start, end, name):
        self.no = no
        self.start = start
        self.end = end
        self.name = name

In [128]:
# 定义节点类  
class Node:
    def __init__(self):
        self.in_channels = []
        self.out_channels = []
        

In [129]:
Chan = []
for i in range(ChanNum):
    chan = Channel(i, None, None, 'channel_'+str(i))
    Chan.append(chan)

In [130]:
sep = '\t'
def parse_line(line):
    global sep
    cols = line.split(sep)
    if len(cols) > 3:
        return cols[:4]
    else:
        return None

In [131]:
# 读取每个河道段数据
fpath = folder + 'TOPO.txt'

print(f'Opening file: {fpath}') 

with open(fpath) as f:
    lines = f.readlines()

    for line in lines:
        data = parse_line(line)
        if data:
            no, start, end, name = data
            print(no, start, end, name) 
print('Handle the relationship between river section number and node number:')

Opening file: /home/louyaxin/1DHydroDynamicModel/input/TOPO.txt
                

1 2 1 NJ_1

2 3 1 NJ_2

3 4 1 NJ_3

Handle the relationship between river section number and node number:


## 子程序

In [132]:
import numpy as np

def tran_chan_junc():
    print(f'ChanNum={ChanNum}') 
    print(f'JuncNum={JuncNum}')
    
    Junc.ki = 1
    Junc.ko = 1
    print('River section number in out')
    
    for i in range(ChanNum):
        s = Chan[i].st 
        e = Chan[i].en
        
        Junc[e].in_list[Junc[e].ki-1] = i
        Junc[s].out_list[Junc[s].ko-1] = i
        
        Junc[e].ki += 1 
        Junc[s].ko += 1
        
        print(f'{Chan[i].no} {s} {e} {Junc[e].ki-1} {Junc[s].ko-1}')
        print(f'{Chan[i].no} {s} {e} ki({e})={Junc[e].ki-1} {Junc[s].ko-1}')

    Junc.ki -= 1
    Junc.ko -= 1
    
    for i in range(1, JuncNum+1):
        print(f'{i} {Junc[i].ki} {Junc[i].ko}')
        
    print('Statistics of river reach at inflow and outflow nodes')
    
    for i in range(1, JuncNum+1):
        Junc[i].flag = 0
        for j in Junc[i].in_list:
            pass
            
        for j in Junc[i].out_list:
            pass
            
    return

In [133]:
#打开时间步长文件
with open(folder + 'TIME.txt') as f:
    dt = float(f.readline())  
    rdt = float(f.readline())

In [134]:
# 根据Junc中的标记分类节点
for i in range(JuncNum1+1, JuncNum2):
    Junc[i].flag = 1
    print(f'Nodes {i} is downstream boundary point')

for i in range(JuncNum2+1, JuncNum):
    Junc[i].flag = -1    
    print(f'Nodes {i} is upstream boundary point')

Nodes 3 is upstream boundary point


In [135]:
Bou = []

if JuncNum1 != 0:
    j = 1
    for i in range(1, JuncNum+1):
        if Junc[i].flag != 0:
            bou_item = {}
            bou_item['No'] = i
            bou_item['flag'] = Junc[i].flag
            
            if Junc[i].flag == 1:
                bou_item['nc'] = Junc[i].out_list[0] 
            #elif Junc[i].flag == -1:
              #  bou_item['nc'] = Junc[i].in_list[0]
                 
            Bou.append(bou_item)
            j += 1
            
if JuncNum1 == 0:
    j = 1
    for i in range(1, JuncNum+1):
        if Junc[i].flag != 0:
            bou_item = {}
            bou_item['No'] = i
            bou_item['flag'] = Junc[i].flag
            
            if Junc[i].flag == 1:
                bou_item['nc'] = Junc[i].out_list[0]  
            elif Junc[i].flag == -1:
                bou_item['nc'] = Junc[i].in_list[0]
            
            Bou.append(bou_item)
            j += 1
            
BouNum = j - 1

print(f'Number of boundary points: {BouNum}') 

for i in range(1, BouNum+1):
    #print(i, Bou[i-1]['No'], Bou[i-1]['flag'],Bou[i-1]['nc'])
    print(i, Bou[i-1]['No'], Bou[i-1]['flag'])
    
print('Done')

Number of boundary points: 1
1 3 -1
Done


In [136]:
# 读取河段属性

Chan = [[] for _ in range(ChanNum)]
nsb = [0] * ChanNum  
nse = [0] * ChanNum
ccc = [0] * ChanNum

with open(folder + 'SECTION.txt') as f:
    for j in range(0,ChanNum):
        Chan[j] = Channel()  
        sum = 0
        Chan[j].name, nsb[j], nse[j] = f.readline().split()
        nsb[j] = int(nsb[j])
        nse[j] = int(nse[j])
        
        for i in range(nsb[j], nse[j]):
            Chan[j].dx[i] = float(f.readline().split()[1])
            sum += Chan[j].dx[i]

        ccc[j] = sum

TypeError: __init__() missing 4 required positional arguments: 'no', 'start', 'end', and 'name'

In [137]:
with open(folder + 'SECTION.txt') as f:
    for j in range(0, ChanNum):
        sum = 0
        Chan[j].name, nsb[j], nse[j] = f.readline().split()
        print(f'reach-section {Chan[j].name} {j} {nsb[j]} {nse[j]}')
        
        for i in range(nsb[j], nse[j]-1):
            kk, Chan[j].dx[i] = f.readline().split()
            sum += Chan[j].dx[i]
            
        ccc[j] = sum

ValueError: not enough values to unpack (expected 3, got 1)

In [138]:
# 读取曼宁系数
with open(folder + 'MANNING.txt') as f:
    for j in range(1, ChanNum):
        for i in range(nsb[j], nse[j]):
            Chan[j].rrn[i], Chan[j].rn[i], Chan[j].lrn[i], Chan[j].mrn[i], Chan[j].hrn[i] = f.readline().split()

In [139]:
# 读取河床高程
se = np.zeros((ChanNum, 2, SecN, SecN))
with open(folder + 'BED_CON.txt') as f, open(folder1 + 'zb.txt', 'w') as f1:
    sno = int(f.readline())
    
    for j in range(1, ChanNum):
        ri, knum1 = f.readline().split()
        
        for i in range(nsb[j], nse[j]):
            knum, sno = f.readline().split()
            
            for k in range(1, sno):
                se[j,1,i,k], se[j,2,i,k] = f.readline().split()
                
            print(f"Number of cross-section layers {j} {i}") 
            Chan[j].zb[i] = min(se[j,2,i,0:sno])
            print(se[j,2,i,0:sno])
            f1.write(f"{j} {i} {Chan[j].zb[i]}\n")
            
print('Terrain condition input completed')

Terrain condition input completed


In [140]:
# 打开上游边界文件
with open(folder + 'BOUNDARY_UP.txt') as f:
    for j in range(1, UbouNum):
        # 读取文件名
        filename = f.readline().strip()
        
        # 打开具体的数据文件
        with open(folder + filename + '.txt') as f2:
        
            i = 1
            flag = 0
            
            # 读取时间和高程数据,直到遇到负时间结束
            while flag == 0:
                t, y = map(float, f2.readline().split())
                Bou[j].t[i-1] = t 
                Bou[j].y[i-1] = y
                
                if t < 0:
                    flag = 1
                else:
                    i += 1
                    
            # 关闭文件
            f2.close()
            
        # 记录数据条数    
        Bou[j].m = i - 1
        print(f'm2={Bou[j].m}')
        
print('Upstream boundary condition loaded')

Upstream boundary condition loaded
